# Đồ Án Cuối Kì : Khoa Học Dữ Liệu ( The Global 2000 - 2022 )
Nhóm sinh viên thực hiện : 

21424068 - Nguyễn Nguyên Văn

21424004 - Nguyễn Thành Đạt

21424061 - Nguyễn Minh Trọng Trí


### Đầu tiên ta thêm các thư viện cũng như import các thư viện cần thiết

In [ ]:
!pip install scrapy
!pip install spider3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import json
import re
import pandas as pd
# import scrapy
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
# from scrapy.selector import Selector

Như mọi người cũng đã biết nhu cầu toàn cầu hoá kinh tế là hết sức quan trọng đối với các công ty đa quốc gia, tập đoàn lớn. Vậy các công ty, <b>các tập đoàn đó là gì?</b> </br>
Chúng ta sẽ thu thập số liệu các công ty đó và <b>đánh giá yếu tố </b> nào đánh giá một công ty, tập đoàn lớn mạnh.</br>


### Tạo project scrapy

In [ ]:
!scrapy startproject fobes_companies_crawler

New Scrapy project 'fobes_companies_crawler', using template directory '/usr/local/lib/python3.10/dist-packages/scrapy/templates/project', created in:
    /content/fobes_companies_crawler

You can start your first spider with:
    cd fobes_companies_crawler
    scrapy genspider example example.com


In [ ]:
cd /content/fobes_companies_crawler/fobes_companies_crawler/spiders

/content/fobes_companies_crawler/fobes_companies_crawler/spiders


## Thu thập dữ liệu
Chúng ta sẽ tạo file .py cần thiết và tiến hành thu thập dữ liệu</br>
Dữ liệu chúng ta thu thập ở https://www.forbes.com/lists/global2000 đây là một trang báo tương đối uy tín rất đáng tin cậy.

In [ ]:
!scrapy crawl get_company -O dataset/get_company.json

## Tiền xử lý dữ liệu
Sau khi đã có dữ liệu ta tiến hành chuyển thành dataframe để dễ dàng xử lý.

In [7]:
df_company = pd.read_json("/content/get_company.json")

Kiểm tra dữ liệu có mấy `hàng` mấy `cột`

In [8]:
df_company.shape

(2000, 7)

Show `dữ liệu` top 10 để xem trước dữ liệu có vấn đề gì không.

In [9]:
df_company.head(10)

,Rank,Name,Country,Sales,Profits,Assets,Market value
0,1,Berkshire Hathaway,United States,$276.09 B,$89.8 B,$958.78 B,$741.48 B
1,2,ICBC,China,$208.13 B,$54.03 B,"$5,518.51 B",$214.43 B
2,3,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,$400.38 B,$105.36 B,$576.04 B,"$2,292.08 B"
3,4,JPMorgan Chase,United States,$124.54 B,$42.12 B,"$3,954.69 B",$374.45 B
4,5,China Construction Bank,China,$202.07 B,$46.89 B,"$4,746.95 B",$181.32 B
5,6,Amazon,United States,$469.82 B,$33.36 B,$420.55 B,"$1,468.4 B"
6,7,Apple,United States,$378.7 B,$100.56 B,$381.19 B,"$2,640.32 B"
7,8,Agricultural Bank of China,China,$181.42 B,$37.38 B,"$4,561.05 B",$133.38 B
8,9,Bank of America,United States,$96.83 B,$31 B,"$3,238.22 B",$303.1 B
9,10,Toyota Motor,Japan,$281.75 B,$28.15 B,$552.46 B,$237.73 B





#### Set Index
Trước tiên ta để cột `Rank` trở thành index của `df_company`



In [10]:
df_company = df_company.set_index("Rank")
df_company.head()

,Name,Country,Sales,Profits,Assets,Market value
Rank,,,,,,
1,Berkshire Hathaway,United States,$276.09 B,$89.8 B,$958.78 B,$741.48 B
2,ICBC,China,$208.13 B,$54.03 B,"$5,518.51 B",$214.43 B
3,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,$400.38 B,$105.36 B,$576.04 B,"$2,292.08 B"
4,JPMorgan Chase,United States,$124.54 B,$42.12 B,"$3,954.69 B",$374.45 B
5,China Construction Bank,China,$202.07 B,$46.89 B,"$4,746.95 B",$181.32 B


#### Ý nghĩa từng cột trong DataFrame
- Theo dữ liệu năm 2022
  - Cột `Rank` : Thứ hạng của các công ty
  - Cột `Name` : Tên của các công ty
  - Cột `Revenue` : Doanh thu của các công ty trong năm
  - Cột `Assets` : Tổng tài sản của các công ty 
  - Cột `Profits` : Lợi nhuận của các công ty trong năm
  - Cột `Market Value` : Giá trị Cng ty đạt được theo định giá trị thị trường (thông qua cổ phiếu)

#### Ý nghĩa của dòng 
- Mỗi 1 dòng trong DataFrame thể hiện <b>doanh thu, tổng tài sản, lợi nhuận, giá trị thị trường</b> trong năm đang xét của một công ty.
- Với giá trị đang xét thông thường sẽ là $B$ (tỉ đô-la), nhưng vẫn có trường đạt $T$( nghìn tỉ đô) và cũng có trường đạt $M$ ( Triệu đô), vì vậy ta cần chuẩn hoá về 1 giá trị ( ta chuẩn hoá về $B$)

Tiếp tục xử lý các giá trị như đã nói ở trên

In [11]:

def transform_cost(value):
    data = value.replace(",","")
    if data[-1] == 'M':  # Triệu chia 1000
      return np.float64(data[1:-2])/1000
    elif data[-1] == 'T': # Nghìn tỷ nhân 1000
      return np.float64(data[1:-2])*1000
    elif data[-1] == 'B':
      return np.float64(data[1:-2])
    return None


In [12]:
df_company["Sales"] = df_company["Sales"].apply(transform_cost)
df_company["Profits"] = df_company["Profits"].apply(transform_cost)
df_company["Assets"] = df_company["Assets"].apply(transform_cost)
df_company["Market value"] = df_company["Market value"].apply(transform_cost)

Đổi tên cột theo phương thức đã nói ở trên

In [13]:
df_company.rename(columns = {'Sales':'Sales (B$)', 'Profits':'Profits (B$)','Assets' : 'Assets (B$)' , 'Market value' : 'Market value (B$)'}, inplace = True)

In [14]:
df_company.head()

,Name,Country,Sales (B$),Profits (B$),Assets (B$),Market value (B$)
Rank,,,,,,
1,Berkshire Hathaway,United States,276.09,89.80,958.78,741.48
2,ICBC,China,208.13,54.03,5518.51,214.43
3,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,400.38,105.36,576.04,2292.08
4,JPMorgan Chase,United States,124.54,42.12,3954.69,374.45
5,China Construction Bank,China,202.07,46.89,4746.95,181.32


**Tổng quan dữ liệu**

Ta xem tổng quan dữ liệu thử có đủ dữ liệu hay không, `xem thử giá trị trung bình, độ lệch chuẩn, giá trị nhỏ nhất, lớn nhất cũng như bộ tứ phân vị của dữ liệu`


In [15]:
df_company.describe()# count = 2000

,Sales (B$),Profits (B$),Assets (B$),Market value (B$)
count,2000.000000,2000.000000,2000.000000,2000.000000
mean,23.875905,2.502110,117.124169,38.252432
std,40.943971,6.159208,363.742939,117.204585
min,0.000000,-12.050000,0.166000,0.144000
25%,5.607500,0.590000,14.797500,7.330000
50%,12.170000,1.055000,32.010000,16.645000
75%,23.725000,2.270000,77.870000,34.710000
max,572.750000,105.360000,5518.510000,2640.320000


Có vẻ dữ liệu của chúng ta đầy đủ (count = 2000)

Tuy nhiên độ lệch của các công ty cũng khá lớn ✌